# Import Libraris

In [ ]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials

import time
from array import array
import os
import shutil
from PIL import Image, ImageDraw
import sys
import time
import requests
from io import BytesIO
import pandas as pd
import numpy as np
from tqdm.auto import tqdm, trange
from glob import glob
import gc
import warnings
warnings.filterwarnings('ignore')

# Get ORG Data

In [ ]:
org_path = "" # 대상자의 폴더들이 들어있는 메인 폴더 경로

# Define Azure Cognitive service Key

In [ ]:
# Azure OCR Resource의 구독 key 값 및 endpoint 경로
subscription_key = ""
endpoint = ""

computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))

# Image Crop Setting

In [ ]:
# bmp 파일의 높이, 길이의 크기에 따른 Shear Rate Table의 위치 Crop 함수
def get_shear_table(org_img, table_path):
    width, height = org_img.size
    if width >= 1800:
        # Setting the points for cropped image
        shear_img = org_img.crop((1050, 310, 1400, 460))
        shear_img.save(table_path)
    else:
        shear_img = org_img.crop((750, 250, 1000, 400))
        shear_img.save(table_path)
    
    read_shear_image = open(table_path,"rb")
    
    return read_shear_image

# Get Y

In [ ]:
# bmp 이미지를 가져와서 OCR 수행하는 함수
def bmp_process(bmp_path):
    # Get Image
    bmp_img = Image.open(bmp_path)
    tmp_path = "/".join(bmp_path.replace("\\","/").split("/")[:-1])
    
    table_path = os.path.join(tmp_path, "img_crop.png")
    
    read_shear_image = get_shear_table(bmp_img, table_path)
        
    # Call API with URL and raw response (allows you to get the operation location)
    read_response = computervision_client.read_in_stream(read_shear_image,  raw=True)
    # Get the operation location (URL with an ID at the end) from the response
    read_operation_location = read_response.headers["Operation-Location"]
    # Grab the ID from the URL
    operation_id = read_operation_location.split("/")[-1]
    
    # Call the "GET" API and wait for it to retrieve the results 
    while True:
        read_result = computervision_client.get_read_result(operation_id)
        if read_result.status not in ['notStarted', 'running']:
            break
        time.sleep(1)

    # Result to DataFrame
    result_list = []
    if read_result.status == OperationStatusCodes.succeeded:
        for text_result in read_result.analyze_result.read_results:
            for line in text_result.lines:
                result_list.append(line.text)
    
    # OCR 결과 중 ","로 인식되었을 경우 "."으로 변환
    re_result_list = []
    for i in range(len(result_list)):
        value = result_list[i]
        value.replace(",",".")
        re_result_list.append(value)

    try:
        result_dict ={
            re_result_list[0] : [float(re_result_list[1])],
            re_result_list[2] : [float(re_result_list[3])],
            re_result_list[4] : [float(re_result_list[5])],
            re_result_list[6] : [float(re_result_list[7])],
            re_result_list[8] : [float(re_result_list[9])],
            re_result_list[10] : [float(re_result_list[11])],
            re_result_list[12] : [float(re_result_list[13])],
            re_result_list[14] : [float(re_result_list[15])],
            re_result_list[16] : [float(re_result_list[17])],
        }
        result_df = pd.DataFrame(result_dict)
    except:
        result_df = pd.DataFrame()
    
    gc.collect()
    return result_df

In [ ]:
# OCR 결과 저장하기 위한 폴더
trg_path = "./Ubiosis/tmp_y"
# 해당 폴더가 미존재 시 폴더 생성
if os.path.isdir(trg_path) == False:
    os.mkdir(trg_path)

root_path = "./Ubiosis/data" # 데이터 폴더 경로

# 1. 학습 대상자 폴더 정합성 확인
## 운영체제 특성 또는 외장하드 특성상 숨김 파일 또는 임시 파일까지 읽어들이는 경우가 있어서 필터링 필요
tmp_subject_list = list(os.listdir(root_path)) # 데이터 저장 폴더 안에 있는 파일 및 폴더명을 불러와서 리스트에 저장
subject_list = [] # 정상 대상자 id 저장할 리스트 객체
for i in range(len(tmp_subject_list)):
    subject = tmp_subject_list[i] # 리스트 객체의 원소 1개씩 탐색
    first_char = subject[0] # 폴더 및 파일 명의 첫번째 문자를 가져옴
    # 비식별화를 수행하였으면, 폴더의 명이 u 로 시작하기 때문에 첫번째 문자가 u 로 시작하는 폴더명을 정상 대상자 id 리스트 객체에 저장
    if first_char == 'u':
        subject_list.append(subject)

err_list = [] # 오류 발생 시 오류가 발생한 대상자 정보 저장하기 위한 리스트 객체
for idx in trange(len(subject_list)):
    # 대상자 ID 정보 호출 후 경로 변수 생성
    subject = subject_list[idx]
    subject_path = os.path.join(org_path, subject)
    
    # 정상 데이터 대상으로 작업 수행
    # Target 폴더에 ID 폴더 생성
    new_id_path = os.path.join(trg_path, subject)
    if os.path.isdir(new_id_path) == False:
        os.mkdir(new_id_path)
    
    org_file_list = list(os.listdir(subject_path))
    for file in org_file_list:
        file_ext = file.split(".")[-1]
        
        if file_ext == "txt":
            shutil.copy(os.path.join(subject_path, file), os.path.join(new_id_path, (subject + "_yprime.txt")))
        elif file_ext == "bmp":
            result_df = bmp_process(os.path.join(subject_path, file))
            
            if len(result_df) > 0:
                result_path = os.path.join(new_id_path, (subject + "_y.csv"))
                result_df.to_csv(result_path, index=False)
            else:
                pass